# MIMIC-III M3Care Baseline

### Load modules and define constants.

In [104]:
from dataset import MIMICDataset
from m3care import M3Care
from comp import MLP
from synth import ( NLEmbedder, TimeSeriesNLEmbedder )
from torch import nn
from vocab import Vocab
from torch.nn import functional as F
import torch
import os

In [2]:
processed_dir = './data/processed'

### Load Datasets

In [3]:
train_ds = MIMICDataset(processed_dir, True)
# test_ds = MIMICDataset(processed_dir, False)

In [4]:
vocab = Vocab.from_json(os.path.join(processed_dir, 'vocab.json'))

In [5]:
DEM_DIM = 18
VIT_DIM = 105
ITV_DIM = 14

### Instantiate Unimodal Extraction Models

In [6]:
DEM_EMB_DIM = 32
ITV_EMB_DIM = 32
VIT_EMB_DIM = 32
NST_EMB_DIM = 256
NTS_EMB_DIM = 512

DROPOUT = 0.3

In [7]:
sample = train_ds[0]
samples = train_ds[:10]

In [8]:
dem_mdl = MLP(in_dim=DEM_DIM, hidden_dim=[256,128,64], out_dim=DEM_EMB_DIM, bias=True, relu=True, norm=True)

In [9]:
vit_mdl = nn.GRU(VIT_DIM, VIT_EMB_DIM*2, 2, batch_first=True, dropout=DROPOUT)

In [10]:
itv_mdl = nn.GRU(ITV_DIM, ITV_EMB_DIM*2, 2, batch_first=True, dropout=DROPOUT)

In [11]:
nst_mdl = NLEmbedder(vocab, 16, NST_EMB_DIM, 8, 1024, dropout=DROPOUT)

In [12]:
nts_mdl = TimeSeriesNLEmbedder(vocab, 16, NTS_EMB_DIM, 8, 2048, dropout=DROPOUT)

### Instantiate M3Care Model

In [105]:
x = torch.ones((1))+2

In [107]:
F.sigmoid(x)

tensor([0.9526])

In [108]:
y = torch.tensor([0,0.5,1,2,3,4])

In [110]:
y > F.sigmoid(x)

tensor([False, False,  True,  True,  True,  True])

In [99]:
for pat_nts in nts:
    times, cats, notes = [torch.tensor(v) for v in pat_nts]
    

In [92]:
filled = torch.zeros(nst_msk.size(0), res.size(1))
filled_cnt = 0
for idx, val in enumerate(nst_msk):
    if val:
        filled[idx] = res[filled_cnt]
        filled_cnt += 1

In [14]:
model = M3Care(dem_mdl, vit_mdl, itv_mdl, nst_mdl, nts_mdl)

TypeError: M3Care.__init__() missing 1 required positional argument: 'device'

In [16]:
model

M3Care(
  (dem_mdl): MLP(
    (model): Sequential(
      (0): Linear(in_features=18, out_features=256, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): ReLU()
      (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): ReLU()
      (8): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): Linear(in_features=64, out_features=32, bias=True)
      (10): ReLU()
      (11): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (vit_mdl): GRU(105, 64, num_layers=2, batch_first=True, dropout=0.3)
  (itv_mdl): GRU(14, 64, num_layers=2, batch_first=True, dropout=0.3)
  (nst_mdl): NLEmbedder(
    (word_embed): WordEmbedder(
      (embedder): Embedding(10003, 16)
      (lin

In [26]:
dem_res = dem_mdl(dem_sample)

In [33]:
lenny = torch.ones((dem_res.size(0), 1)).int().squeeze()

In [37]:
max_len = lenny.max().item()

In [41]:
mask = torch.arange(max_len).int().expand(len(lenny), max_len)

In [45]:
dem_res.shape

torch.Size([10, 32])

In [54]:
torch.ones(dem_res.size(0), 1) == 1

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]])

In [17]:
train_ds[0:1][3]

[(array([  0,   3,   5,  13,  14,  15,  24,  26,  27,  36,  37,  38,  50,
          50,  50,  50,  51,  62,  64,  65,  74,  76,  86,  86,  86,  86,
          91,  97,  99, 110, 110, 110, 110, 110, 110, 110, 110, 110, 119,
         121, 136]),
  array([[0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 

In [34]:
sample = train_ds[0:2]

In [35]:
sample[5]

array([[ True, False],
       [ True, False]])

In [38]:
sample[5]

array([[ True, False],
       [ True, False]])

In [42]:
sample[3][1]

(array([ 0,  0,  0,  0,  1,  3, 11, 18, 24, 24, 24, 24, 24, 24, 24, 24, 24,
        37, 48]),
 array([[0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0]]),
 array([[15126,  3543,     4, ...,  6280,  5535,     4],
        [15126,  3543,     4, ...,  6280,  5535,     4],
        [  174,  2002,     4, ...,     0,     0,     0],
        ...,
        [  174,    97,     4, ...,     0,     0,     0],
        [ 1021,  

In [6]:
res = ds[0:10]

In [7]:
res[3]

[<Closed HDF5 dataset>, <Closed HDF5 dataset>]

In [8]:
with open

array([[ True, False],
       [ True, False],
       [ True,  True],
       [ True, False],
       [False, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True]])